## Helper function to clean and split directors and countries

In [ ]:
import re

def clear_and_split_text(text):
    """
    Cleans and processes the input text by removing numerical references in square brackets,
    normalizing line breaks, and concatenating lines into a single string separated by commas.
    """
    text = re.sub(r'\[\d+\]', '\n', text)

    text = re.sub(r'\n+', '\n', text).strip()
    texts = text.split('\n')
    res = texts[0]
    for t in texts[1:]:
        res += ", " + t

    return res



# 🎬 Web Scraping: Highest-Grossing Films

## Overview
This script scrapes **Wikipedia's List of Highest-Grossing Films** to extract key details about top box-office movies, including:

- 🎥 **Title**
- 📅 **Year of Release**
- 💰 **Box Office Earnings**
- 🔗 **Wikipedia Link**
- 🎬 **Director**
- 🌍 **Country of Origin**

## How It Works
- Uses `requests` to fetch the Wikipedia page.
- Parses the HTML using `BeautifulSoup` to extract film data from a table.
- Retrieves additional details (director & country) from each film’s page.
- Stores the results in a structured format JSON.


In [50]:
import requests
from bs4 import BeautifulSoup
import json
import time

URL = "https://en.wikipedia.org/wiki/List_of_highest-grossing_films"
HEADERS = {"User-Agent": "Mozilla/5.0"}

def get_film_data():
    """
    Scrapes the Wikipedia page to retrieve a list of the highest-grossing films,
    including their title, release year, box office earnings, Wikipedia link, director, and country.

    Returns:
    list: A list of dictionaries, each containing details about a film.
    """
    response = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")

    table = soup.find("table", {"class": "wikitable"})
    rows = table.find_all("tr")[1:]  
    
    films = []
    for row in rows:
        cols = row.find_all(["th", "td"])

        title_tag = cols[2].find("a")
        title = title_tag.text.strip() if title_tag else cols[2].text.strip()
        link = "https://en.wikipedia.org" + title_tag["href"] if title_tag else None
        

        year = int(cols[4].text.strip())
        

        box_office = int(extract_digits(cols[3].text.strip().replace("$", "").replace(",", "")))
        

        director, country = get_film_details(link) if link else (None, None, None)
        
        films.append({
            "title": title,
            "year": year,
            "box_office": box_office,
            "link": link,
            "director": director,
            "country": country
        })
        

    
    return films

def get_film_details(film_url):

    """
    Extracts additional details about a film, such as director and country of origin,
    from its Wikipedia page.

    Parameters:
    film_url (str): The Wikipedia URL of the film.

    Returns:
    tuple: (director, country) - The director and country information.
    """
    
    response = requests.get(film_url, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")
    
    info_box = soup.find("table", {"class": "infobox"})
 
    director = clear_and_split_text(extract_info(info_box, "Directed by"))
    country = clear_and_split_text(extract_info(info_box, ["Country", "Countries"]))

    
    return director, country


def extract_info(info_box, label):
    """
    Extracts specific information from a Wikipedia infobox.

    Parameters:
    info_box (BeautifulSoup object): The infobox HTML element.
    label (str or list): The label(s) of the information to extract.

    Returns:
    str: The extracted text, if found; otherwise, None.
    """
    
    row = info_box.find("th", string=label)
    if row:
        td = row.find_next_sibling("td")
        return td.text.strip() if td else None
    return None


film_data = get_film_data()



In [ ]:
# Save in JSON
with open("highest_grossing_films.json", "w", encoding="utf-8") as f:
    json.dump(film_data, f, indent=4, ensure_ascii=False)


# 🎬 Storing Highest-Grossing Films in SQLite

## Overview
This script:
- Creates a **SQLite database** (`highest_films.db`) if it doesn't exist.
- Defines a table **`films`** to store movie details.
- Reads **film data from JSON**.
- Inserts the extracted data into the SQLite database.

## Database Schema
The table **`films`** has the following structure:
- 🆔 `id` - Unique ID (Primary Key)
- 🎥 `title` - Film Title
- 📅 `release_year` - Year of Release
- 🎬 `director` - Director Name
- 💰 `box_office` - Box Office Revenue
- 🌍 `country` - Country of Origin



In [61]:
import sqlite3



DB_FILE = "highest_films.db"


conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()


cursor.execute('''
CREATE TABLE IF NOT EXISTS films (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    release_year INTEGER,
    director TEXT,
    box_office TEXT,
    country TEXT
)
''')
conn.commit()


with open("highest_grossing_films.json", "r", encoding="utf-8") as f:
    film_data = json.load(f)


films_to_insert = [
    (
        film["title"],
        int(film["year"]),
        film["director"],
        film["box_office"],
        film["country"]
    )
    for film in film_data
]


cursor.executemany("""
    INSERT INTO films (title, release_year, director, box_office, country)
    VALUES (?, ?, ?, ?, ?)
""", films_to_insert)
conn.commit()
conn.close()




## Check DB

In [63]:
conn = sqlite3.connect("highest_films.db")
cursor = conn.cursor()

# Fetch and display data
cursor.execute("SELECT * FROM films")
rows = cursor.fetchall()
for row in rows[:5]:
    print(row)

conn.close()

(1, 'Avatar', 2009, 'James Cameron', '2923706026', 'United Kingdom, United States')
(2, 'Avengers: Endgame', 2019, 'Anthony RussoJoe Russo', '2797501328', 'United States')
(3, 'Avatar: The Way of Water', 2022, 'James Cameron', '2320250281', 'United States')
(4, 'Titanic', 1997, 'James Cameron', '2257844554', 'United States')
(5, 'Star Wars: The Force Awakens', 2015, 'J. J. Abrams', '2068223624', 'United States')
